## Dicionário de dados

- '**receita_cliente**': Renda do cliente em R$
- '**anuidade_emprestimo**': Valor anual da taxa de juros do empréstimo em $
- '**anos_casa_propria**': Idade da propriedade do cliente em anos
- '**telefone_trab**': Acessibilidade do número de telefone comercial (1 indica Sim e 0 indica Não)
- '**avaliacao_cidade**': Classificação da cidade do cliente: 3 para excelente, 2 para bom e 1 para médio.
- '**score_1**': Pontuação originada de uma fonte externa. Este é um escore normalizado.
- '**score_2**': Pontuação originada de uma fonte externa. Este é um escore normalizado.
- '**score_3**': Pontuação originada de uma fonte externa. Este é um escore normalizado.
- '**score_social**': Quantidade de amigos/familiares do cliente que não cumpriram com pagamentos de empréstimos nos últimos 60 dias.
- '**troca_telefone**': Quantidade de dias antes do pedido de empréstimo em que o cliente mudou seu número de telefone.
- '**inadimplente**': 1 indica que o cliente não honrou com o pagamento do empréstimo, e 0 indica o contrário.

## Carregando os dados

In [1]:
import pandas as pd

dados = pd.read_csv("https://3070-classificacao-otimizacao.s3.us-east-2.amazonaws.com/dados_inadimplencia.csv")
dados.head()

,receita_cliente,anuidade_emprestimo,anos_casa_propria,telefone_trab,avaliacao_cidade,score_1,score_2,score_3,score_social,troca_telefone,inadimplente
0,16855.246324,2997.000000,12.157324,0,2.0,0.501213,0.003109,0.513171,0.117428,243.0,1
1,13500.000000,2776.050000,12.157324,0,2.0,0.501213,0.269730,0.513171,0.097900,617.0,0
2,11250.000000,2722.188351,12.157324,0,3.0,0.701396,0.518625,0.700184,0.118600,9.0,0
3,27000.000000,6750.000000,3.000000,0,2.0,0.501213,0.649571,0.513171,0.047400,300.0,0
4,22500.000000,3097.800000,12.157324,0,2.0,0.440744,0.509677,0.513171,0.014400,2913.0,1


In [2]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14578 entries, 0 to 14577
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   receita_cliente      14578 non-null  float64
 1   anuidade_emprestimo  14578 non-null  float64
 2   anos_casa_propria    14578 non-null  float64
 3   telefone_trab        14578 non-null  int64  
 4   avaliacao_cidade     14578 non-null  float64
 5   score_1              14578 non-null  float64
 6   score_2              14578 non-null  float64
 7   score_3              14578 non-null  float64
 8   score_social         14578 non-null  float64
 9   troca_telefone       14578 non-null  float64
 10  inadimplente         14578 non-null  int64  
dtypes: float64(9), int64(2)
memory usage: 1.2 MB


In [3]:
dados["inadimplente"].value_counts(normalize=True)*100

inadimplente
0    67.649883
1    32.350117
Name: proportion, dtype: float64

## Dividindo os dados em treino e teste

In [4]:
from sklearn.model_selection import train_test_split

SEED = 42

x = dados.drop("inadimplente", axis=1)
y = dados["inadimplente"]

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.33, stratify=y)

## Criando os modelos

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score

modelo_decision_tree = DecisionTreeClassifier(max_depth=3, random_state=SEED)
modelo_decision_tree.fit(x_treino, y_treino)
y_pred = modelo_decision_tree.predict(x_teste)
print(f"Recall Decision Tree: {recall_score(y_teste, y_pred)}")

Recall Decision Tree: 0.13688946015424164


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

regression_pipeline = make_pipeline(StandardScaler(), LogisticRegression())
regression_pipeline.fit(x_treino, y_treino)
y_pred = regression_pipeline.predict(x_teste)
print(f"Recall Logistic Regression: {recall_score(y_teste, y_pred)}")

Recall Logistic Regression: 0.22300771208226222


## GridSearchCV

### Decision Tree

In [7]:
import numpy as np
from sklearn.model_selection import GridSearchCV, StratifiedKFold

param_grid_dt = {
    'criterion':  ['gini', 'entropy'],
    'max_depth': np.linspace(6, 12, 4, dtype=int),
    'min_samples_split': np.linspace(5, 20, 4, dtype=int),
    'min_samples_leaf': np.linspace(5, 20, 4, dtype=int),
    'max_features': ['sqrt', 'log2'],
    'splitter': ['best', 'random']
}


decision_tree = DecisionTreeClassifier(max_depth=3, random_state=SEED)
cv = StratifiedKFold(shuffle=True, random_state=SEED)

tree_search_cv = GridSearchCV(
    decision_tree, param_grid=param_grid_dt, cv=cv, scoring="recall", n_jobs=-1)

tree_search_cv.fit(x_treino, y_treino)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(max_depth=3, random_state=42),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 6,  8, 10, 12]),
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': array([ 5, 10, 15, 20]),
                         'min_samples_split': array([ 5, 10, 15, 20]),
                         'splitter': ['best', 'random']},
             scoring='recall')

In [8]:
tree_search_cv.best_params_

{'criterion': 'entropy',
 'max_depth': 12,
 'max_features': 'sqrt',
 'min_samples_leaf': 5,
 'min_samples_split': 20,
 'splitter': 'best'}

In [9]:
df_tree_search_cv = pd.DataFrame(tree_search_cv.cv_results_)
df_tree_search_cv.head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008532,0.002448,0.002869,0.001448,gini,6,sqrt,5,5,best,"{'criterion': 'gini', 'max_depth': 6, 'max_fea...",0.245253,0.212025,0.172468,0.145570,0.280063,0.211076,0.048394,197
1,0.004085,0.001460,0.004017,0.002007,gini,6,sqrt,5,5,random,"{'criterion': 'gini', 'max_depth': 6, 'max_fea...",0.153481,0.155063,0.181962,0.134494,0.148734,0.154747,0.015419,489
2,0.010956,0.002465,0.003434,0.001875,gini,6,sqrt,5,10,best,"{'criterion': 'gini', 'max_depth': 6, 'max_fea...",0.245253,0.212025,0.172468,0.145570,0.280063,0.211076,0.048394,197


In [13]:
df_tree_search_cv.loc[tree_search_cv.best_index_]

mean_fit_time                                                       0.015956
std_fit_time                                                        0.004582
mean_score_time                                                     0.002818
std_score_time                                                      0.001441
param_criterion                                                      entropy
param_max_depth                                                           12
param_max_features                                                      sqrt
param_min_samples_leaf                                                     5
param_min_samples_split                                                   20
param_splitter                                                          best
params                     {'criterion': 'entropy', 'max_depth': 12, 'max...
split0_test_score                                                   0.245253
split1_test_score                                                   0.299051

### Análise visual

In [14]:
import plotly.express as px

px.scatter(df_tree_search_cv, x="param_max_depth", y="mean_test_score", title="Max Depth vs. Recall")

In [15]:
px.scatter(df_tree_search_cv, x="param_criterion", y="mean_test_score", title="Criterion vs. Recall")

In [16]:
px.scatter(df_tree_search_cv, x="param_max_features", y="mean_test_score", title="Max Features vs. Recall")

In [17]:
px.scatter(df_tree_search_cv, x="param_min_samples_split", y="mean_test_score", title="Min. Samples Split vs. Recall")